# Introducción
https://metadata.fundacionsadosky.org.ar/competition/20/

Una “oportunidad” consiste en un proyecto de venta o instalación de equipos para un cliente. La venta se estructura alrededor de TRF (Toneladas de refrigeración) y puede estar compuesta por varios productos distintos. El "pipeline" hace referencia al flujo de oportunidades prospecto que la empresa está desarrollando. El equipo comercial asigna a distintos momentos, para cada oportunidad, un estado en la negociación. En la Ilustración se muestran los estados que las oportunidades tienen dentro del pipeline.

La variable que se está tratando de predecir es “Probabilidad de éxito” para cada oportunidad. ¿Cuál es la probabilidad de que la oportunidad se convierta en un caso Closed Won?

#### Evaluación

La calificación de la competencia se logra, en un 50%, con el resultado de las predicciones. La métrica elegida para el set de validación es la función de Logistic Loss (Log Loss). https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss

En el tablero se puede ver la evaluación de la solución propuesta en una muestra del set de validación. Esta muestra puede variar durante la competencia. El total del conjunto de datos de validación se reserva hasta el final de la competencia.

El 50% restante se obtiene presentando un informe donde se deberá explicar qué tratamiento hicieron en los datos y una explicación sobre el modelo propuesto. La nota la otorga un jurado ad-hoc de especialistas en el tema evaluando el informe.

Las predicciones se deben enviar en un archivo CSV sin encabezado conteniendo solamente la identificación de la oportunidad (Opportunity_ID) del dataset de valuación y la columna de predicciones (score). A continuación, se muestran tres filas de un archivo de predicciones típico:

Opportunity_ID, score

10689, 0.33

10690, 0.9853289

10691, 0.70956707

etc.

In [1]:
# Importo librerías a utillizar
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss

In [2]:
# Leo el archivo de entrenamiento
df = pd.read_csv('Entrenamieto_ECI_2020.csv', index_col ='ID')
df

,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
ID,,,,,,,,,,,,,,,,,,,,,
27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,None,...,Q2,2016,NaT,10,EUR,5272800.00,EUR,5272800.0,Closed Lost,Prod_Category_A_None
27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,None,...,Q1,2016,NaT,0,EUR,48230.00,EUR,48230.0,Closed Won,Prod_Category_A_None
27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,Source_7,...,Q1,2016,NaT,0,USD,83865.60,USD,83865.6,Closed Won,Prod_Category_A_None
16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,...,Q1,2018,NaT,14,USD,7421881.50,USD,7421881.5,Closed Lost,Prod_Category_A_None
16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,...,Q1,2018,NaT,25,USD,13357192.50,USD,13357192.5,Closed Lost,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8781,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,Source_7,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
8786,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,Source_7,...,Q2,2016,NaT,0,EUR,93015.00,EUR,299715.0,Closed Won,Prod_Category_A_None
8792,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,Source_7,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None


# Limpieza de datos

In [3]:
df.loc[((df['Territory'] == 'None') & (df['Region'] == 'Japan')), 'Territory'] = 'Japan'

In [4]:
df_temp = df[df['Territory'] == 'None']['Region'].value_counts()
df_temp

Americas       571
EMEA           187
Middle East     30
Name: Region, dtype: int64

## Quitar campos que definí como no relevantes para el modelo
Genero una lista de campos que considero como no relevantes por diversos motivos:
- Fechas que tienen demasiados registros para ser considerado como un campo con info consistente.
- Variables con la mayoría de sus valores 'None'.
- Variables con la mayoría de los registros en un valor determinado.
- No existe un patrón identificable en el valor de cada oportunidad.
- Baja confiabilidad en el dato. Ejemplo: 'Planned_Delivery_Start_Date'. De las 16k oportunidades listadas existen 986 fechas únicas en esta variable. Mi supuesto (a confirmar con FríoFrío) es que las personas que cargan las oportunidades lo ven como un campo a llenar sin demasiada rigurosidad. Otras variables con este "inconveniente" Planned_Delivery_End_Date, Last_Modified_Date, Last_Modified_By, Opportunity_Created_Date, Quote_Expiry_Date



In [5]:
# Aquí defino la lista dropped_features para incluir todas las columnas que DESCARTARÉ para el modelo. 
# La justificación de droppear cada una está en FrioFrio_Features.ipynb
dropped_features = ['Territory',
                    'Submitted_for_Approval', 
                    'Account_Created_Date',
                    'Billing_Country',
                    'Opportunity_Name', 
                    'Opportunity_ID', 
                    'Sales_Contract_No', 
                    'Quote_Type', 
                    'Opportunity_Created_Date',
                    'Brand', 
                    'Product_Type', 
                    'Size', 
                    'Product_Category_B', 
                    'Price', 
                    'Currency', 
                    'Last_Activity', 
                    'Quote_Expiry_Date',
                    'Last_Modified_Date', 
                    'Last_Modified_By', 
                    'ASP_Currency', 
                    'ASP', 
                    'ASP_(converted)_Currency', 
                    'Planned_Delivery_Start_Date', 
                    'Planned_Delivery_End_Date', 
                    'Month',
                    'Delivery_Year', 
                    'Actual_Delivery_Date',
                    'Total_Amount',
                    'Total_Amount_Currency',
                    'Total_Taxable_Amount_Currency', 
                    'Prod_Category_A', 
 ]

In [6]:
df = df.drop(columns=dropped_features)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16947 entries, 27761 to 28318
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Region                              16947 non-null  object 
 1   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 2   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 3   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 4   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 5   Bureaucratic_Code                   16947 non-null  object 
 6   Source                              16947 non-null  object 
 7   Account_Name                        16947 non-null  object 
 8   Account_Owner                       16947 non-null  object 
 9   Opportunity_Owner                   16947 non-null  object 
 10  Account_Type                        16947 non-null  object 
 11  Opportunity_Type                    1

## Corregir los dtypes de variables
Corrijo los dtypes de algunas variables según la info publicada en https://metadata.fundacionsadosky.org.ar/competition/20/#participate . Principalmente las fechas las convierto en datetime.

In [ ]:
# Modifico a datetime las variables que son fechas. Sean utilizadas o no.

# df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'])
# df1['Opportunity_Created_Date'] = pd.to_datetime(df1['Opportunity_Created_Date'])
# df['Last_Activity'] = pd.to_datetime(df['Last_Activity'])
# df1['Quote_Expiry_Date'] = pd.to_datetime(df1['Quote_Expiry_Date'])
# df['Last_Modified_Date'] = pd.to_datetime(df['Last_Modified_Date'])
# df1['Planned_Delivery_Start_Date'] = pd.to_datetime(df1['Planned_Delivery_Start_Date'])
# df1['Planned_Delivery_End_Date'] = pd.to_datetime(df1['Planned_Delivery_End_Date'])
# df['Actual_Delivery_Date'] = pd.to_datetime(df['Actual_Delivery_Date'])

# df1.info()

## Eliminar las oportunidades que están activas
Dado que no conozco el resultado de las oportunidades activas las elimino de los datos que utilizaré para crear el modelo de predicción.

In [7]:
df = df[(df['Stage'] == 'Closed Lost') | (df['Stage'] == 'Closed Won')]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16883 entries, 27761 to 28318
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Region                              16883 non-null  object 
 1   Pricing, Delivery_Terms_Quote_Appr  16883 non-null  int64  
 2   Pricing, Delivery_Terms_Approved    16883 non-null  int64  
 3   Bureaucratic_Code_0_Approval        16883 non-null  int64  
 4   Bureaucratic_Code_0_Approved        16883 non-null  int64  
 5   Bureaucratic_Code                   16883 non-null  object 
 6   Source                              16883 non-null  object 
 7   Account_Name                        16883 non-null  object 
 8   Account_Owner                       16883 non-null  object 
 9   Opportunity_Owner                   16883 non-null  object 
 10  Account_Type                        16883 non-null  object 
 11  Opportunity_Type                    1

## Eliminar las 112 oportunidades Account_Type = 'None'
Existen 112 oportunidades que no tienen un Account_Type. Me gustaría conocer como se asigna esta variable. Dado que son menos del 1% de los registros los eliminaré del modelado.

In [8]:
df = df[df['Account_Type'] != 'None']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16771 entries, 27761 to 28318
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Region                              16771 non-null  object 
 1   Pricing, Delivery_Terms_Quote_Appr  16771 non-null  int64  
 2   Pricing, Delivery_Terms_Approved    16771 non-null  int64  
 3   Bureaucratic_Code_0_Approval        16771 non-null  int64  
 4   Bureaucratic_Code_0_Approved        16771 non-null  int64  
 5   Bureaucratic_Code                   16771 non-null  object 
 6   Source                              16771 non-null  object 
 7   Account_Name                        16771 non-null  object 
 8   Account_Owner                       16771 non-null  object 
 9   Opportunity_Owner                   16771 non-null  object 
 10  Account_Type                        16771 non-null  object 
 11  Opportunity_Type                    1

## ASP_(converted)

In [9]:
# elimino los ASP mayores a 1 y los menores a 0.1 por Outliers
df = df[(df['ASP_(converted)']<1) & (df['ASP_(converted)']>0.1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13261 entries, 27761 to 28318
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Region                              13261 non-null  object 
 1   Pricing, Delivery_Terms_Quote_Appr  13261 non-null  int64  
 2   Pricing, Delivery_Terms_Approved    13261 non-null  int64  
 3   Bureaucratic_Code_0_Approval        13261 non-null  int64  
 4   Bureaucratic_Code_0_Approved        13261 non-null  int64  
 5   Bureaucratic_Code                   13261 non-null  object 
 6   Source                              13261 non-null  object 
 7   Account_Name                        13261 non-null  object 
 8   Account_Owner                       13261 non-null  object 
 9   Opportunity_Owner                   13261 non-null  object 
 10  Account_Type                        13261 non-null  object 
 11  Opportunity_Type                    1

## Delivery_Time (variable creada) (NO UTILIZADO)
Creo la variable df['Delivery_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date'] y elimino las otras dos del calculo.

In [ ]:
# df1['Delivery_Time'] = df1['Planned_Delivery_End_Date']-df1['Planned_Delivery_Start_Date']
# df1['Delivery_Time'] = df1['Delivery_Time'].dt.components.days
# df1 = df1[(df1['Delivery_Time']<100) 
# #           & (df1['Delivery_Time']<0)
#          ]
# df1 = df1.drop(columns=['Planned_Delivery_End_Date', 'Planned_Delivery_Start_Date'])
# df1.info()

## Convertir `Stage` en binaria
Utilizo la siguiente convención 1 = 'Closed Won' y 0 = 'Closed Lost'

In [10]:
df['Stage'] = df['Stage'].replace('Closed Won', 1)
df['Stage'] = df['Stage'].replace('Closed Lost', 0)
df['Stage']

ID
27761    0
27760    1
27446    1
16808    0
16805    0
        ..
8781     1
8786     1
8792     1
28561    0
28318    0
Name: Stage, Length: 13261, dtype: int64

# Dummies o Encoder

- Pasar las variables a dummies
- Separar en X e y
- Separar en Train y Test con train_test_split
- Elegir modelo
- Cargar el modelo con X_train e y_train y Fittearlo
- Predecir el modelo con los valores de X_test. Predict
- Comparar el resultado de la predicción con los valores y_test

In [11]:
# - Separar en X e y
X = df.drop(columns=['Stage'])
y = df['Stage']

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()

In [ ]:
# dummy_variables = list(X.select_dtypes(include='object').columns)
# for dummy in dummy_variables:
#     le.fit(X[dummy])
#     X[dummy] = le.transform(X[dummy])
# X.head()

In [12]:
# - Pasar las variables a dummies

# Estas son las variables que debo pasar a dummies/Hotencodear
# dummy_variables = list(df1.select_dtypes(include='object').columns)
# dummy_variables

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore',
                    sparse= False                    
#     categories= dummy_variables
    )
X1= ohe.fit_transform(X)
X1

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [13]:
# - Separar en Train y Test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X1, y, test_size=0.2, random_state=42)

# Modelos

## LinearRegression

In [ ]:
# - Elegir el modelo y fittearlo
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
# - Predecir el valor de X_test
y_pred_LinearRegression = lr.predict(X_test)
y_pred_LinearRegression

In [ ]:
# - Evaluar con log_loss
log_loss(y_test, y_pred_LinearRegression)

## Support Vector Regression

In [14]:
from sklearn import svm

In [ ]:
svr = svm.SVR(kernel = 'poly',
              degree=5,
              gamma= 'scale',
              C=1
             )
svr.fit(X_train,y_train)
y_pred_svr = svr.predict(X_test)
y_pred_svr

In [ ]:
svr.score(X_test, y_test)

In [ ]:
log_loss(y_test, y_pred_svr)

## Linear SVR

In [ ]:
from sklearn.svm import LinearSVR

In [ ]:
linsvr = LinearSVR()
linsvr.fit(X_train, y_train)
y_pred_linsvr = linsvr.predict(X_test)
y_pred_linsvr

In [ ]:
log_loss(y_test, y_pred_linsvr).round(5)

## NuSVR

In [ ]:
from sklearn.svm import NuSVR

In [ ]:
nusvr = NuSVR()
nusvr.fit(X_train, y_train)
y_pred_nusvr = nusvr.predict(X_test)
y_pred_nusvr

In [ ]:
log_loss(y_test, y_pred_nusvr).round(5)

## Logistic Regression

In [ ]:
log_loss(y_test, y_pred_svr).round(5)

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(max_depth=5, random_state=0)
rfr.fit(X_train,y_train)
y_pred_rfr = rfr.predict(X_test)
y_pred_rfr

In [ ]:
log_loss(y_test, y_pred_rfr)

# Evaluación del modelo

In [ ]:
y_pred

In [ ]:
y_test

# Predicción de mi modelo

- Leer `Validacion_ECI_2020.csv`
- Codificar las variables con los mismos pasos que `Entrenamieto.csv`
- Predecir en un `y_pred` el resultado de utilizar mi modelo en `X_val`

In [ ]:
df_val = pd.read_csv('Validacion_ECI_2020.csv', index_col='ID')
df_val

In [ ]:
df_val = df_val.drop(columns=dropped_features)
df_val.info()

In [ ]:
df_val = df_val.fillna(df_val['ASP_(converted)'].mean())

In [ ]:
X_val = ohe.transform(df_val)

In [ ]:
# Indicar cual es el modelo elegido para la predicción. Utilizar el que tiene menor log_loss de los analizados.
model = svm

In [ ]:
y_val_pred = model.predict(X_val)
y_val_pred

In [ ]:
d = {'Opportunity_ID': df_val.index, 'score': y_val_pred}
output = pd.DataFrame(d)
output.index = output['Opportunity_ID']
output.to_csv('pmolteni_prediccion.csv', index=False)

# Anexo

In [ ]:
from sklearn.svm import LinearSVC
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# dropped_features.append('Stage')
# X = df1.drop(columns=['Stage'])
# y = df1['Stage']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fijo los parámetros de Linear SVC para que Grid Search combine y evalue 
parameters = {'C':[0.001, 0.01, 0.1, 1, 10],
                'loss':['hinge', 'squared_hinge']}

# Defino la variable con el modelo a utilizar
linearsvc = LinearSVC()

# Cargo grid_search con el modelo (linearsvc), el diccionario con los parámetros del SVC a evaluar (param_grid), CV=5 para que realice 5 cortes en la información a entrenar, scoring define el modo de evaluar el gridsearch con el área bajo la curva, return_train_score devuelve los resultados del train
grid_search = GridSearchCV(linearsvc, param_grid= parameters, scoring = 'roc_auc' , cv=5, return_train_score=True) 

# Entreno el modelo con X_Train e y_train.
grid_search.fit(X_train, y_train)

In [ ]:
# Quiero ver los resultados del mean_train_score de cada combinación que realizó GridSearch utilizando el modelo LinearSVC
sorted(grid_search.cv_results_['mean_train_score'])

In [ ]:
# Quiero ver los resultados del mean_test_score de cada combinación que realizó GridSearch utilizando el modelo LinearSVC
sorted(grid_search.cv_results_['mean_test_score'])

In [ ]:
# quiero conocer los mejores parámetros para SVC que hay encontrado mi grid Search 
grid_search.best_params_

In [ ]:
# quiero conocer los mejores parámetros para SVC que hay encontrado mi grid Search 
grid_search.best_params_